In [1]:
import torch
from jetbot import Robot

In [10]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print(device, torch.cuda.get_device_name(0))
else:
    device= torch.device("cpu")
    print(device)

cuda NVIDIA Tegra X1


In [34]:
data = torch.zeros((1, 22))
data

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [35]:
model = torch.load("George-1-128.pth")
model.eval()
model.to(device)

SeparateActorCritic(
  (network_body): NetworkBody(
    (visual_processors): ModuleList()
    (vector_processors): ModuleList(
      (0): VectorInput(
        (normalizer): Normalizer()
      )
    )
    (linear_encoder): LinearEncoder(
      (seq_layers): Sequential(
        (0): Linear(in_features=22, out_features=128, bias=True)
        (1): Swish()
      )
    )
  )
  (distribution): GaussianDistribution(
    (mu): Linear(in_features=128, out_features=2, bias=True)
  )
  (critic): ValueNetwork(
    (network_body): NetworkBody(
      (visual_processors): ModuleList()
      (vector_processors): ModuleList(
        (0): VectorInput(
          (normalizer): Normalizer()
        )
      )
      (linear_encoder): LinearEncoder(
        (seq_layers): Sequential(
          (0): Linear(in_features=22, out_features=128, bias=True)
          (1): Swish()
        )
      )
    )
    (value_heads): ValueHeads(
      (value_heads): ModuleDict(
        (extrinsic): Linear(in_features=128, out_fea

In [46]:
hidden,_ = model.network_body(vis_inputs=[0],vec_inputs=[data.to(device)])
distribution = model.distribution(hidden)
action = distribution.sample()

In [47]:
print(action)

tensor([[2.3724, 5.6187]], grad_fn=<AddBackward0>)


In [48]:
action = (action).detach().cpu().numpy()
action

array([[2.3723712, 5.618699 ]], dtype=float32)

In [52]:
forward = action[0][0]/5
angle = action[0][1]/5

forward,angle

(0.4744742393493652, 1.1237398147583009)

In [50]:
import time
robot = Robot()

robot.forward(0.5)
time.sleep(forward)
robot.stop()

In [53]:
robot.right(0.5)
time.sleep(angle)
robot.stop()

In [6]:
results,_ = model.critic(vec_inputs=[data], vis_inputs=[0])

In [7]:
print(results)

{'extrinsic': tensor([-2.9442], grad_fn=<SqueezeBackward1>)}
